In [1]:
import pandas as pd
from useful_functions import fix_me
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nameparser import HumanName

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
ucla = pd.read_csv('../../Data/Rosters/UCLA_Health_2020-11-01.csv')
ppd = pd.read_csv('../../Data/PPD/ppd_data_20201121.csv')
spec = pd.read_csv('../../Data/PPD/speciality_id.csv')
emails = pd.read_csv('../../Data/Email/emails_20201130.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def split_names(roster_df):
    '''Splits name column into components'''
    roster_df = roster_df.drop_duplicates()
    dict_list = []
    for row in roster_df.itertuples():
        name_parsed = HumanName(row.name)
        name_dict = {
            'name': row.name,
            'FIRST_NAME': name_parsed.first.upper(),
            'LAST_NAME': name_parsed.last.upper(),
            'MIDDLE_NAME': name_parsed.middle.upper(),
            'SUFFIX': name_parsed.suffix.upper(),
            'NICKNAME': name_parsed.nickname.upper(),
            'TITLE': name_parsed.title.upper(),
        }
        dict_list.append(name_dict)
    name_df = pd.DataFrame(dict_list)
    new_df = pd.merge(name_df, roster_df, on='name')
    return new_df

In [4]:
ucla = split_names(ucla)
ppd['ME'] = fix_me(ppd.ME)
emails['ME'] = fix_me(emails.ME)
PPD = pd.merge(emails, ppd, on='ME', how='left')

In [16]:
spec = spec[['SPEC_CD','DESC']]

In [19]:
PPD = ppd[['ME', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MD_DO_CODE',
       'TELEPHONE_NUMBER',  'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MEDSCHOOL_STATE', 'MEDSCHOOL_GRAD_YEAR', 
       'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2',
       'POLO_CITY', 'POLO_STATE', 'POLO_ZIP']]

In [15]:
UCLA = ucla[ucla.degree.isin(['MD','DO'])]

In [20]:
PPD = pd.merge(PPD, spec, left_on='PRIM_SPEC_CD', right_on='SPEC_CD')
PPD = pd.merge(PPD, spec, left_on='SEC_SPEC_CD', right_on='SPEC_CD', suffixes = ['_PRIMARY', '_SECONDARY'])

In [22]:
PPD = PPD[['ME', 'MAILING_NAME', 'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME',
       'SUFFIX', 'MD_DO_CODE', 'TELEPHONE_NUMBER', 'MEDSCHOOL_STATE', 'MEDSCHOOL_GRAD_YEAR',
       'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE',
       'POLO_ZIP', 'DESC_PRIMARY', 
       'DESC_SECONDARY']]

In [28]:
PPD['DEGREE'] = ['MD' if x == 1 else 'DO' for x in PPD.MD_DO_CODE]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,TELEPHONE_NUMBER,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE
0,00102000013,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,1,NaN,1,2000,NaN,NaN,NaN,NaN,NaN,EMERGENCY MEDICINE,FAMILY MEDICINE,MD
1,00102700040,MERRILL R BARRON MD,BARRON,MERRILL,RAY,NaN,1,3.346992e+09,1,1970,ALLSOUTH URGENT CARE,4550 W MAIN ST,DOTHAN,AL,36305,EMERGENCY MEDICINE,FAMILY MEDICINE,MD
2,00102700066,DAVID M BRICKMAN MD,BRICKMAN,DAVID,MICHAEL,NaN,1,NaN,1,1970,NaN,NaN,NaN,NaN,NaN,EMERGENCY MEDICINE,FAMILY MEDICINE,MD
3,00102710223,STANLEY J GALLE JR MD,GALLE,STANLEY,JOS,2.0,1,NaN,1,1971,NaN,7700 S BROADWAY,LITTLETON,CO,80122,EMERGENCY MEDICINE,FAMILY MEDICINE,MD
4,00102741129,CECIL L ENNIS MD,ENNIS,CECIL,LAWRENCE,NaN,1,NaN,1,1974,NaN,750 MORPHY AVE,FAIRHOPE,AL,36532,EMERGENCY MEDICINE,FAMILY MEDICINE,MD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281362,02301921791,GEOFFREY L ROSENTHAL MD,ROSENTHAL,GEOFFREY,LAHN,NaN,1,4.103284e+09,23,1992,UNIV OF MARYLAND HOSPITAL FOR CHILDREN,110 S PACA ST FL 7,BALTIMORE,MD,21201,PEDIATRIC CARDIOLOGY,ADULT CONGENITAL HEART DISEASE (INTERNAL MEDIC...,MD
1281363,03201990458,MICHAEL E WILLERS MD,WILLERS,MICHAEL,EDWIN,NaN,1,4.133153e+09,32,1999,THE CHILDRENS HEART CENTER,1754 NORTHAMPTON ST,HOLYOKE,MA,1040,PEDIATRIC CARDIOLOGY,ADULT CONGENITAL HEART DISEASE (INTERNAL MEDIC...,MD
1281364,03546071471,IRA B TAUB MD,TAUB,IRA,BARRY,NaN,1,NaN,35,2007,NaN,23250 CHAGRIN BLVD STE 325,CLEVELAND,OH,44122,PEDIATRIC CARDIOLOGY,ADULT CONGENITAL HEART DISEASE (INTERNAL MEDIC...,MD
1281365,04201761102,JUAN VILLAFANE MD,VILLAFANE,JUAN,NaN,NaN,1,NaN,42,1976,NaN,731 E BROADWAY,LOUISVILLE,KY,40202,PEDIATRIC CARDIOLOGY,ADULT CONGENITAL HEART DISEASE (INTERNAL MEDIC...,MD


In [41]:
matched = pd.merge(UCLA, PPD, left_on=['FIRST_NAME', 'LAST_NAME','degree','med_school_grad_year'], right_on = ['FIRST_NAME', 'LAST_NAME','DEGREE','MEDSCHOOL_GRAD_YEAR'])

In [37]:
PPD['MEDSCHOOL_GRAD_YEAR'] = PPD.MEDSCHOOL_GRAD_YEAR.astype(str)

In [39]:
len(UCLA)

3102

In [47]:
unmatched = UCLA[UCLA.name.isin(matched.name)==False].drop_duplicates('name')

In [46]:
len(UCLA.drop_duplicates('name'))

2598

In [50]:
more_match = pd.merge(unmatched, PPD, left_on=['FIRST_NAME', 'LAST_NAME','degree'], right_on = ['FIRST_NAME', 'LAST_NAME','DEGREE'])

In [57]:
still_unmatched = unmatched[unmatched.name.isin(more_match.name)==False]

In [55]:
len(more_match.drop_duplicates('name'))

137

In [56]:
len(unmatched.drop_duplicates('name'))

403

In [64]:
still_unmatched.to_csv('../../Data/Rosters/UCLA_TEST.csv', index=False)

In [63]:
PPD[PPD.LAST_NAME=='ZAVAHIR']

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,TELEPHONE_NUMBER,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE
209884,22003960033,ZAINUL R ZAVAHIR MD,ZAVAHIR,ZAINUL,RIQUAZA,NaN,1,3.108274e+09,220,1996,NaN,4560 ADMIRALTY WAY STE 100,MARINA DL REY,CA,90292,INTERNAL MEDICINE,UNSPECIFIED,MD
1157746,22001690322,MOHAMED F ZAVAHIR MD,ZAVAHIR,MOHAMED,FEIZAL,NaN,1,8.146960e+09,220,1969,NaN,1720 N JUNIATA ST,HOLLIDAYSBURG,PA,16648,PEDIATRICS,ALLERGY,MD


In [99]:
PPD[(PPD.LAST_NAME=='ILARDE')&(PPD.DEGREE=='MD')]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,TELEPHONE_NUMBER,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE
432099,01643952820,EMELIE J ILARDE MD,ILARDE,EMELIE,JONSON,NaN,1,7.084256e+09,16,1996,NaN,4301 W 95TH ST,OAK LAWN,IL,60453,FAMILY MEDICINE,UNSPECIFIED,MD
788618,74802840202,ALDO A ILARDE MD,ILARDE,ALDO,ARRASTIA,NaN,1,8.056426e+09,748,1984,NaN,6633 TELEPHONE RD STE 200,VENTURA,CA,93003,"ENDOCRINOLOGY, DIABETES & METABOLISM",UNSPECIFIED,MD


In [218]:
PPD[(PPD.FIRST_NAME=='JESSICA')&(PPD.DEGREE=='MD')&(PPD.MEDSCHOOL_GRAD_YEAR=='2012')][100:150]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,TELEPHONE_NUMBER,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE
465814,03515120725,JESSICA L FERGER MD,FERGER,JESSICA,LEIGH,NaN,1,6.032295e+09,35,2012,NaN,253 PLEASANT ST,CONCORD,NH,03301,FAMILY MEDICINE,UNSPECIFIED,MD
466919,03546121797,JESSICA J KARP MD,KARP,JESSICA,JILL,NaN,1,2.068704e+09,35,2012,NaN,26401 PACIFIC HWY S,DES MOINES,WA,98198,FAMILY MEDICINE,UNSPECIFIED,MD
468549,03601120304,JESSICA L HART MD,HART,JESSICA,LEE,NaN,1,NaN,36,2012,UNIVERSITY OF COLORADO WARDENBURG HEALTH,1900 WARDENBURG DR,BOULDER,CO,80309,FAMILY MEDICINE,UNSPECIFIED,MD
475787,03901101565,JESSICA M LANDT MD,LANDT,JESSICA,MIKI,NaN,1,4.057931e+09,39,2012,NaN,2900 S TELEPHONE RD STE 200,MOORE,OK,73160,FAMILY MEDICINE,UNSPECIFIED,MD
480711,04113121594,JESSICA A LEE MD,LEE,JESSICA,AMANDA,NaN,1,7.173671e+09,41,2012,NaN,1701 CORNWALL RD STE 101,LEBANON,PA,17042,FAMILY MEDICINE,UNSPECIFIED,MD
481531,04114120826,JESSICA K HENSEL MD,HENSEL,JESSICA,KATHLEEN,NaN,1,NaN,41,2012,NaN,2800 E AJO WAY,TUCSON,AZ,85713,FAMILY MEDICINE,UNSPECIFIED,MD
482109,04115111863,JESSICA A LINK MD,LINK,JESSICA,ANN,NaN,1,NaN,41,2012,NaN,6 HEALTHY WAY,ELLENVILLE,NY,12428,FAMILY MEDICINE,UNSPECIFIED,MD
498402,05107110111,JESSICA L BERNENS MD,BERNENS,JESSICA,L,NaN,1,NaN,51,2012,NaN,1419 CEDAR RD STE 101,CHESAPEAKE,VA,23322,FAMILY MEDICINE,UNSPECIFIED,MD
509036,27513120015,JESSICA C STANDISH MD,STANDISH,JESSICA,CAITLIN,NaN,1,NaN,275,2012,NaN,2500 ALHAMBRA AVE,MARTINEZ,CA,94553,FAMILY MEDICINE,UNSPECIFIED,MD
513300,39647120010,JESSICA A ROUX BRAURE MD,ROUX BRAURE,JESSICA,ANITA YVETTE,NaN,1,NaN,396,2012,NaN,7900 LEES SUMMIT RD,KANSAS CITY,MO,64139,FAMILY MEDICINE,UNSPECIFIED,MD


In [422]:
PPD[(PPD.LAST_NAME=="ABDAbdelnour")&(PPD.DEGREE=='MD')&(PPD.MEDSCHOOL_GRAD_YEAR=='2008')]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,TELEPHONE_NUMBER,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE
736855,02405080292,LUKASZ MACYSZYN MD,MACYSZYN,LUKASZ,NaN,NaN,1,NaN,24,2008,DEPARTMENT OF NEUROSURGERY,300 STEIN PLAZA SUITE 540,LOS ANGELES,CA,90095,NEUROLOGICAL SURGERY,UNSPECIFIED,MD


In [421]:
PPD[(PPD.FIRST_NAME=="KATINA")&(PPD.DEGREE=='MD')&(PPD.MEDSCHOOL_GRAD_YEAR=='2008')]
# .sort_values('LAST_NAME')[30:]
#&(PPD.DESC_PRIMARY=='ANESTHESIOLOGY')

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,TELEPHONE_NUMBER,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE
444250,02301081131,KATINA N MOORE MD,MOORE,KATINA,NICOLE,NaN,1,4.432637e+09,23,2008,NaN,10 HOPKINS PLZ,BALTIMORE,MD,21201,FAMILY MEDICINE,UNSPECIFIED,MD
469689,03607080665,KATINA D ROBERTSON MD,ROBERTSON,KATINA,DENISE,NaN,1,4.043773e+09,36,2008,NaN,200 E PONCE DE LEON AVE STE 110,DECATUR,GA,30030,FAMILY MEDICINE,UNSPECIFIED,MD


In [406]:
PPD[(PPD.DEGREE=='MD')&(PPD.LAST_NAME=="NARAIN")]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,TELEPHONE_NUMBER,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE
64980,01106160054,PUNYA O NARAIN MD,NARAIN,PUNYA,OLGA,NaN,1,NaN,11,2016,CHILDREN'S HOSPITAL LOS ANGELES,4650 W SUNSET BLVD MS 68,LOS ANGELES,CA,90027,PEDIATRICS,UNSPECIFIED,MD
68750,01611954341,TULIKA NARAIN MD,NARAIN,TULIKA,NaN,NaN,1,6.317308e+09,16,1996,SV PEDIATRICS,155 E WOODSIDE AVE,PATCHOGUE,NY,11772,PEDIATRICS,UNSPECIFIED,MD
114727,49505950126,SWATI NARAIN MD,NARAIN,SWATI,NaN,NaN,1,NaN,495,1995,NaN,1 BROOKDALE PLZ,BROOKLYN,NY,11212,PEDIATRICS,UNSPECIFIED,MD
117062,49607800234,BINDOO NARAIN MD,NARAIN,BINDOO,NaN,NaN,1,3.523653e+09,496,1980,NaN,1070 FLAGLER AVE,LEESBURG,FL,34748,PEDIATRICS,UNSPECIFIED,MD
224368,49529940121,GURINDER P NARAIN MD,NARAIN,GURINDER,PAL,NaN,1,5.598756e+09,495,1994,NaN,800 N ST,SANGER,CA,93657,INTERNAL MEDICINE,UNSPECIFIED,MD
225270,49537900091,SUDHIR M NARAIN MD,NARAIN,SUDHIR,M,NaN,1,5.028528e+09,495,1990,NaN,201 ABRAHAM FLEXNER WAY STE 1001,LOUISVILLE,KY,40202,INTERNAL MEDICINE,UNSPECIFIED,MD
287611,04107931118,HITESH NARAIN MD,NARAIN,HITESH,NaN,NaN,1,6.317587e+09,41,1993,NaN,155 E WOODSIDE AVE,PATCHOGUE,NY,11772,OBSTETRICS & GYNECOLOGY,UNSPECIFIED,MD
306482,85001760012,HAROLD NARAIN MD,NARAIN,HAROLD,NaN,NaN,1,NaN,850,1976,TOWNE HOUSE 111,19 NORTH CT,PITTSBURGH,PA,15237,OBSTETRICS & GYNECOLOGY,UNSPECIFIED,MD
337684,49554820050,PRASHANT K NARAIN MD,NARAIN,PRASHANT,KISHORE,NaN,1,6.232470e+09,495,1982,NaN,13555 W MCDOWELL RD STE 204,GOODYEAR,AZ,85395,GENERAL SURGERY,UNSPECIFIED,MD
395183,02307171701,ANKUR S NARAIN MD,NARAIN,ANKUR,SHAH,NaN,1,NaN,23,2018,DEPT OF ORTHOPEDICS,55 LAKE AVE N,WORCESTER,MA,1655,ORTHOPEDIC SURGERY,UNSPECIFIED,MD


In [121]:
ppd[(ppd.FIRST_NAME=='MC DONALD')&(ppd.MD_DO_CODE==2)&(ppd.MEDSCHOOL_GRAD_YEAR==2011)&(ppd.BIRTH_STATE=='CA')]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
81299,00576110791,A,1,2.0,SUSAN A COX DO,COX,SUSAN,ANNE,NaN,NaN,...,CA,90045,5631.0,C035,NaN,NaN,NaN,NaN,NaN,NaN
81363,00576111444,A,1,2.0,SUSAN TRAN DO,TRAN,SUSAN,NaN,NaN,NaN,...,CA,95121,1794.0,C019,NaN,NaN,NaN,NaN,NaN,NaN


In [397]:
PPD[(PPD.FIRST_NAME=='JULIO')&(PPD.LAST_NAME=='MEZA')&(PPD.DEGREE=='MD')]

,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MD_DO_CODE,TELEPHONE_NUMBER,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE


In [220]:
emails[emails.ME=='00514070237']

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME,DOMAIN
169568,00514070237,jrexach@mednet.ucla.edu,AMA Web Site,jrexach,mednet.ucla.edu
169569,00514070237,jrexach@mednet.ucla.edu,Morning Rounds ...,jrexach,mednet.ucla.edu
169570,00514070237,jrexach@mednet.ucla.edu,Morning Rounds Daily Edition ...,jrexach,mednet.ucla.edu
169571,00514070237,jrexach@mednet.ucla.edu,No Usage,jrexach,mednet.ucla.edu
169572,00514070237,jrexach@mednet.ucla.edu,Preferred E-mail,jrexach,mednet.ucla.edu


In [423]:
schools = pd.read_excel('../../Data/Rosters/US Schools Crosswalk AMA_AAMC.xlsx')

In [424]:
schools

,INST_ID,INST_TYPE_CD,INST_NAME_120,AMA_ID
0,848,MSUS,Cooper Medical School of Rowan University,3307
1,854,MSUS,Charles E. Schmidt College of Medicine at Flor...,1108
2,867,MSUS,TCU and UNTHSC School of Medicine,4818
3,872,MSUS,New York University Long Island School of Medi...,3530
4,850,MSUS,University of South Carolina School of Medicin...,4505
...,...,...,...,...
182,844,MSUS,Oakland University William Beaumont School of ...,2517
183,851,MSUS,University of Arizona College of Medicine - Ph...,302
184,846,MSUS,Geisinger Commonwealth School of Medicine,4116
185,845,MSUS,Donald and Barbara Zucker School of Medicine a...,3550


In [427]:
ppd.groupby('MEDSCHOOL_ID').count()

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
MEDSCHOOL_ID,,,,,,,,,,,,,,,,,,,,,
0,47,47,47,47,47,47,47,21,2,16,...,32,32,30,32,0,0,0,0,0,0
1,280654,280654,280654,274448,280654,280645,280650,226920,10225,55347,...,210767,211406,207913,208859,68141,56462,68142,8128,6391,8128
2,150072,150072,150072,148002,150072,150066,150070,125251,5309,26812,...,113850,113952,112387,112856,36233,31048,36234,4034,3298,4034
3,53793,53793,53793,52637,53793,53791,53793,42398,1384,10254,...,40532,40577,39818,40038,13991,11099,13991,1726,1273,1726
4,64605,64605,64605,63640,64605,64605,64605,54405,1955,11943,...,48945,48969,48233,48481,16054,13540,16054,1832,1491,1832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,227,227,227,224,227,227,227,67,1,44,...,190,190,188,190,38,18,38,7,5,7
96,757,757,757,749,757,757,757,480,0,154,...,621,621,619,619,146,121,146,16,11,16
97,256,256,256,252,256,256,256,131,0,41,...,192,192,191,192,53,38,53,5,3,5


In [483]:
states = pd.read_csv('../../Data/Email/state.csv')

In [438]:
pd.merge(matched, states, left_on='MEDSCHOOL_STATE', right_on='SRC_STATE_CD')

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [439]:
matched.dtypes

name                       object
FIRST_NAME                 object
LAST_NAME                  object
MIDDLE_NAME_x              object
SUFFIX_x                   object
NICKNAME                   object
TITLE                      object
link                       object
degree                     object
specialty                  object
secondary_specialty        object
department                 object
parent_org_name            object
med_school                 object
med_school_grad_year       object
residency_program          object
program                    object
location_name              object
location_address           object
location_address_city      object
location_address_state     object
location_address_zip       object
phone                      object
fax                        object
state_license_number       object
ME                         object
MAILING_NAME               object
MIDDLE_NAME_y              object
SUFFIX_y                  float64
MD_DO_CODE    

In [450]:
test = pd.read_csv("../../Data/Rosters/UCLA_TEST.csv")

In [452]:
TEST = test[(test.me!='None')&(test.me!='NONE')]

In [453]:
TEST['ME']=fix_me(TEST.me)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [455]:
TEST = pd.merge(PPD, TEST, on='ME')

KeyError: "['LAST_NAME', 'FIRST_NAME'] not in index"

In [468]:
z = matched[['name', 'MIDDLE_NAME_x', 'SUFFIX_x',
       'NICKNAME', 'TITLE', 'link', 'degree', 'specialty',
       'secondary_specialty', 'department', 'parent_org_name', 'med_school',
       'med_school_grad_year', 'residency_program', 'program', 'location_name',
       'location_address', 'location_address_city', 'location_address_state',
       'location_address_zip', 'phone', 'fax', 'state_license_number', 'ME',
       'MAILING_NAME', 'MIDDLE_NAME_y', 'SUFFIX_y', 'MD_DO_CODE',
       'TELEPHONE_NUMBER', 'MEDSCHOOL_STATE', 'MEDSCHOOL_GRAD_YEAR',
       'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE',
       'POLO_ZIP', 'DESC_PRIMARY', 'DESC_SECONDARY', 'DEGREE']]

In [467]:
y = TEST[['name', 'MIDDLE_NAME_x', 'SUFFIX_x',
       'NICKNAME', 'TITLE', 'link', 'degree', 'specialty',
       'secondary_specialty', 'department', 'parent_org_name', 'med_school',
       'med_school_grad_year', 'residency_program', 'program', 'location_name',
       'location_address', 'location_address_city', 'location_address_state',
       'location_address_zip', 'phone', 'fax', 'state_license_number', 'ME',
       'MAILING_NAME', 'MIDDLE_NAME_y', 'SUFFIX_y', 'MD_DO_CODE',
       'TELEPHONE_NUMBER', 'MEDSCHOOL_STATE', 'MEDSCHOOL_GRAD_YEAR',
       'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE',
       'POLO_ZIP', 'DESC_PRIMARY', 'DESC_SECONDARY', 'DEGREE']]

In [466]:
x = more_match[['name', 'MIDDLE_NAME_x', 'SUFFIX_x',
       'NICKNAME', 'TITLE', 'link', 'degree', 'specialty',
       'secondary_specialty', 'department', 'parent_org_name', 'med_school',
       'med_school_grad_year', 'residency_program', 'program', 'location_name',
       'location_address', 'location_address_city', 'location_address_state',
       'location_address_zip', 'phone', 'fax', 'state_license_number', 'ME',
       'MAILING_NAME', 'MIDDLE_NAME_y', 'SUFFIX_y', 'MD_DO_CODE',
       'TELEPHONE_NUMBER', 'MEDSCHOOL_STATE', 'MEDSCHOOL_GRAD_YEAR',
       'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2', 'POLO_CITY', 'POLO_STATE',
       'POLO_ZIP', 'DESC_PRIMARY', 'DESC_SECONDARY', 'DEGREE']]

In [472]:
all_that = pd.concat([x,y,z])

In [475]:
that_all = all_that.drop_duplicates(['name','ME'])

In [479]:
pd.merge(that_all, states, left_on='MEDSCHOOL_STATE', right_on='STATE_ID')

,name,MIDDLE_NAME_x,SUFFIX_x,NICKNAME,TITLE,link,degree,specialty,secondary_specialty,department,...,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE,SRC_STATE_CD,STATE_ID
0,Ming Guo,,,,,https://www.uclahealth.org/ming-guo,MD,"Neurology, Neurodevelopmental Disabilities",None,Neurology,...,UCLA MEDICAL CENTER,300 MEDICAL PLAZA,LOS ANGELES,CA,90095,INTERNAL MEDICINE,UNSPECIFIED,MD,ID,243
1,Ming Guo,,,,,https://www.uclahealth.org/ming-guo,MD,"Neurology, Neurodevelopmental Disabilities",None,Neurology,...,NaN,1515 HOLCOMBE BLVD,HOUSTON,TX,77030,CYTOPATHOLOGY,UNSPECIFIED,MD,ID,243
2,Tao He,,,,,https://www.uclahealth.org/tao-he,MD,"Medicine, Pulmonary Disease",None,Medicine,...,DEPT PULM & CCM,145 COMMUNITY DR,GREAT NECK,NY,11021,PULMONARY & CRITICAL CARE MEDICINE,UNSPECIFIED,MD,ID,243
3,Peifeng Hu,,,,,https://www.uclahealth.org/peifeng-hu,MD,"Medicine, Geriatric Medicine",None,Medicine,...,UCLA,10945 LE CONTE AVE STE 2339,LOS ANGELES,CA,90024,GERIATRIC MEDICINE (INTERNAL MEDICINE),UNSPECIFIED,MD,ID,243
4,Jason F. Lee,F.,,,,https://www.uclahealth.org/jason-lee,MD,Family Medicine,None,Medicine,...,COLLOM & CARNEY CLINIC,5002 COWHORN CREEK RD,TEXARKANA,TX,75503,NEPHROLOGY,UNSPECIFIED,MD,ID,243
5,Lisa K. Lee,K.,,,,https://www.uclahealth.org/lisa-lee,MD,Anesthesiology,None,Anesthesiology,...,CAREMOUNT MEDICAL PC,600 WESTAGE BUSINESS CTR DR,FISHKILL,NY,12524,ANATOMIC/CLINICAL PATHOLOGY,UNSPECIFIED,MD,ID,243
6,Jennifer Li,,,,,https://www.uclahealth.org/jennifer-li,MD,"Surgery, Transplant",None,Surgery,...,NaN,7232 ROSEMEAD BLVD STE 301,SAN GABRIEL,CA,91775,INTERNAL MEDICINE,UNSPECIFIED,MD,ID,243
7,Ruihong Luo,,,,,https://www.uclahealth.org/ruihong-luo,MD,"Medicine, Infectious Disease",None,Medicine,...,INFECTIOUS DISEASES,850 HARRISON AVE,BOSTON,MA,02118,INFECTIOUS DISEASE,UNSPECIFIED,MD,ID,243
8,Jianyu Rao,,,,,https://www.uclahealth.org/jian-yu-rao,MD,Anatomic Pathology,None,Pathology,...,UCLA MEDICAL CENTER,10833 LE CONTE AVE,LOS ANGELES,CA,90095,CYTOPATHOLOGY,UNSPECIFIED,MD,ID,243
9,Jing Yang,,,,,https://www.uclahealth.org/jing-yang,MD,Pathology,Anatomic Pathology,Pathology,...,NaN,NaN,NaN,NaN,NaN,ANATOMIC/CLINICAL PATHOLOGY,UNSPECIFIED,MD,ID,243


In [478]:
states

,SRC_STATE_CD,STATE_ID
0,~,-1
1,AK,195
2,AL,196
3,CT,197
4,FL,198
...,...,...
60,CZ,254
61,DE,255
62,NJ,256
63,PA,257


In [486]:
that_all.groupby('MEDSCHOOL_STATE').count()[50:100]

,name,MIDDLE_NAME_x,SUFFIX_x,NICKNAME,TITLE,link,degree,specialty,secondary_specialty,department,...,TELEPHONE_NUMBER,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE
MEDSCHOOL_STATE,,,,,,,,,,,,,,,,,,,,,
65,13,13,11,11,11,13,13,13,13,13,...,10,13,9,13,13,12,13,13,13,13
67,7,7,6,6,6,7,7,7,7,7,...,5,7,6,7,7,7,7,7,7,7
68,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,0,0,1,1,1
130,3,3,3,3,3,3,3,3,3,3,...,0,3,3,3,3,3,3,3,3,3
132,9,8,8,8,8,9,9,9,9,9,...,8,9,7,9,9,9,9,9,9,9
143,7,7,7,7,7,7,7,7,7,7,...,6,7,6,7,7,7,7,7,7,7
154,4,4,4,4,4,4,4,4,4,4,...,4,4,3,4,4,4,4,4,4,4
165,2,2,2,2,2,2,2,2,2,2,...,1,2,1,2,2,2,2,2,2,2
187,5,5,4,4,4,5,5,5,5,5,...,3,5,2,5,5,5,5,5,5,5


In [484]:
states

,SRC_STATE_CD,STATE_ID,DESC,SRC_SYS,INSERT_TMSTP,UPDATE_TMSTP,INSERT_USERID,UPDATE_USERID,AMA_STATE_CD
0,~,-1,NOT AVAILABLE / UNKNOWN,MANUAL,13AUG2015:00:00:00.000000,13AUG2015:00:00:00.000000,MANUAL,MANUAL,NaN
1,AK,195,Alaska,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,2.0
2,AL,196,Alabama,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,1.0
3,CT,197,Connecticut,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,8.0
4,FL,198,Florida,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,11.0
...,...,...,...,...,...,...,...,...,...
60,CZ,254,Canal Zone,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,NaN
61,DE,255,Delaware,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,9.0
62,NJ,256,New Jersey,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,33.0
63,PA,257,Pennsylvania,MASTERFILE,16DEC2005:03:31:35.000000,16DEC2005:14:50:48.000000,ETL_LOAD,ETL_LOAD,41.0


In [487]:
that_all

,name,MIDDLE_NAME_x,SUFFIX_x,NICKNAME,TITLE,link,degree,specialty,secondary_specialty,department,...,MEDSCHOOL_STATE,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE
0,Jonathan G. Goldin,G.,,,,https://www.uclahealth.org/jonathan-goldin,MD,Diagnostic Radiology,None,Radiological Sciences,...,836,1983,BL428/CHS,10833 LE CONTE AVE,LOS ANGELES,CA,90095,DIAGNOSTIC RADIOLOGY,UNSPECIFIED,MD
1,Jonathan W. Goldman,W.,,,,https://www.uclahealth.org/jonathan-goldman,MD,"Medicine, Hematology and Oncology",None,Medicine,...,24,2010,NaN,148 CHESTNUT ST,NEEDHAM,MA,2492,INTERNAL MEDICINE,UNSPECIFIED,MD
2,Jonathan W. Goldman,W.,,,,https://www.uclahealth.org/jonathan-goldman,MD,"Medicine, Hematology and Oncology",None,Medicine,...,5,2001,UCLA,2020 SANTA MONICA BLVD STE 600,SANTA MONICA,CA,90404,HEMATOLOGY/MEDICAL ONCOLOGY,UNSPECIFIED,MD
3,Jonathan W. Goldman,W.,,,,https://www.uclahealth.org/jonathan-goldman,MD,"Medicine, Hematology and Oncology",None,Medicine,...,917,1989,NaN,NaN,NaN,NaN,NaN,CARDIOVASCULAR DISEASE,INTERNAL MEDICINE,MD
4,Wayne W. Grody,W.,,,,https://www.uclahealth.org/wayne-grody,MD,Pediatrics,"Pathology, Molecular Genetic",Pathology,...,48,1977,DEPT OF PATHOLOGY - UCLA,10833 LE CONTE AVE,LOS ANGELES,CA,90095,MEDICAL GENETICS,MOLECULAR GENETIC PATHOLOGY (PATHOLOGY AND MED...,MD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2597,Erik N. Zeegen,N.,,,,https://www.uclahealth.org/erik-zeegen,MD,"Orthopaedic Surgery, Adult Reconstructive Surg...",Orthopaedic Surgery,Orthopaedic Surgery,...,5,1996,NaN,1250 16TH ST STE 3145,SANTA MONICA,CA,90404,ORTHOPEDIC SURGERY,MUSCULOSKELETAL ONCOLOGY,MD
2598,Michelle R. Zeidler,R.,,,,https://www.uclahealth.org/michelle-zeidler,MD,"Medicine, Pulmonary Disease",None,Medicine,...,5,1995,UCLA PULMONERY CRITICAL CR,10833 LE CONTE AVE,LOS ANGELES,CA,90095,PULMONARY & CRITICAL CARE MEDICINE,CRITICAL CARE MEDICINE (INTERNAL MEDICINE),MD
2599,William Zeiger,,,,,https://www.uclahealth.org/william-zeiger,MD,Neurology,None,Neurology,...,16,2013,NaN,710 WESTWOOD PLZ STE 1240,LOS ANGELES,CA,90095,NEUROLOGY,UNSPECIFIED,MD
2600,Melanie R. Zermeno,R.,,,,https://www.uclahealth.org/melanie-zermeno,MD,Adult Psychiatry,None,Psychiatry,...,5,2003,NaN,300 UCLA MEDICAL PLZ STE 2200,LOS ANGELES,CA,90095,PSYCHIATRY,UNSPECIFIED,MD


In [500]:
def match_spec(spec_1, spec_2):
    match = False
    spec_1 = spec_1.upper()
    spec_2 = spec_2.upper()
    if spec_1 in spec_2:
        match=True
    elif spec_2 in spec_1:
        match=True
    else:
        score = fuzz.ratio(spec_1, spec_2)
        if score>45:
            match=True
        print(score)
    return(match)

In [502]:
match_list =[]
for row in that_all.itertuples():
    print('..')
    spec_1 = row.specialty
    spec_2 = row.DESC_PRIMARY
    print(spec_1)
    print(spec_2)
    match = match_spec(spec_1, spec_2)
    print(match)
    if match == False:
        print(row.DESC_SECONDARY)
        print(row.secondary_specialty)
    match_list.append(match)

..
Diagnostic Radiology
DIAGNOSTIC RADIOLOGY
True
..
Medicine, Hematology and Oncology
INTERNAL MEDICINE
32
False
UNSPECIFIED
None
..
Medicine, Hematology and Oncology
HEMATOLOGY/MEDICAL ONCOLOGY
67
True
..
Medicine, Hematology and Oncology
CARDIOVASCULAR DISEASE
22
False
INTERNAL MEDICINE
None
..
Pediatrics
MEDICAL GENETICS
62
True
..
Pathology
ANATOMIC/CLINICAL PATHOLOGY
True
..
Neurology, Neurodevelopmental Disabilities
INTERNAL MEDICINE
31
False
UNSPECIFIED
None
..
Neurology, Neurodevelopmental Disabilities
CYTOPATHOLOGY
18
False
UNSPECIFIED
None
..
Internal Medicine
PEDIATRICS
30
False
UNSPECIFIED
None
..
Internal Medicine
CHILD & ADOLESCENT PSYCHIATRY
17
False
UNSPECIFIED
None
..
Anesthesiology
PAIN MEDICINE
22
False
UNSPECIFIED
None
..
Neurology
ANATOMIC/CLINICAL PATHOLOGY
33
False
UNSPECIFIED
Neurology, Vascular Neurology
..
Medicine, Pulmonary Disease
PULMONARY & CRITICAL CARE MEDICINE
43
False
UNSPECIFIED
None
..
Ophthalmology
UNSPECIFIED
8
False
UNSPECIFIED
None
..
Internal 

Medicine, Geriatric Medicine
CHILD & ADOLESCENT PSYCHIATRY
28
False
UNSPECIFIED
None
..
Medicine, Cardiology / Cardiovascular Disease
INTERNAL MEDICINE
26
False
UNSPECIFIED
None
..
Anatomic Pathology
INTERNAL MEDICINE
29
False
UNSPECIFIED
None
..
Anatomic Pathology
ORTHOPEDIC SURGERY
39
False
UNSPECIFIED
None
..
Anatomic Pathology
FAMILY MEDICINE
24
False
UNSPECIFIED
None
..
Anatomic Pathology
NEUROLOGY
44
False
UNSPECIFIED
None
..
Anatomic Pathology
ANATOMIC/CLINICAL PATHOLOGY
80
True
..
Anatomic Pathology
VASCULAR SURGERY
29
False
AEROSPACE MEDICINE
None
..
Medicine, Cardiology / Cardiovascular Disease
UNSPECIFIED
7
False
UNSPECIFIED
None
..
Adult Psychiatry
UNSPECIFIED
7
False
UNSPECIFIED
None
..
Medicine, Pulmonary Disease
PULMONARY & CRITICAL CARE MEDICINE
43
False
UNSPECIFIED
None
..
Surgery, Complex General Surgical Oncology
SURGICAL ONCOLOGY
True
..
Neurology
NEUROLOGY
True
..
Neurology
NEUROLOGY
True
..
Psychiatry
PSYCHIATRY
True
..
Neurology, Neurodevelopmental Disabilities
I

None
..
Anesthesiology
ANESTHESIOLOGY
True
..
Diagnostic and Interventional Radiology
DIAGNOSTIC RADIOLOGY
68
True
..
Medicine, Infectious Disease
INFECTIOUS DISEASE
True
..
Anesthesiology
ANESTHESIOLOGY
True
..
Dermatology
PEDIATRIC DERMATOLOGY
True
..
Urology
UROLOGY
True
..
Ophthalmology
OPHTHALMOLOGY
True
..
Dermatology
DERMATOLOGY
True
..
Medicine, Gastroenterology
GASTROENTEROLOGY
True
..
Otolaryngology / Head and Neck
OTOLARYNGOLOGY-HEAD AND NECK SURGERY
82
True
..
Obstetrics and Gynecology
MATERNAL AND FETAL MEDICINE
38
False
UNSPECIFIED
None
..
Neurological Surgery
NEUROLOGICAL SURGERY
True
..
Medicine, Cardiology / Cardiovascular Disease
CARDIOVASCULAR DISEASE
True
..
Diagnostic Radiology, Neuroradiology
NEURORADIOLOGY
True
..
Diagnostic Radiology
DIAGNOSTIC RADIOLOGY
True
..
Medicine, Cardiology / Cardiovascular Disease
CARDIOVASCULAR DISEASE
True
..
Medicine, Pulmonary Disease
PULMONARY & CRITICAL CARE MEDICINE
43
False
UNSPECIFIED
Medicine, Critical Care Medicine
..
Medici

PULMONARY & CRITICAL CARE MEDICINE
43
False
UNSPECIFIED
None
..
Diagnostic Radiology, Neuroradiology
NEURORADIOLOGY
True
..
Medicine, Hematology and Oncology
ORTHOPEDIC SURGERY
24
False
UNSPECIFIED
None
..
Medicine, Hematology and Oncology
FAMILY MEDICINE
33
False
UNSPECIFIED
None
..
Medicine, Hematology and Oncology
ANESTHESIOLOGY
38
False
UNSPECIFIED
None
..
Surgery
UNSPECIFIED
22
False
UNSPECIFIED
Medicine, Critical Care Medicine
..
Neurology
NEUROLOGY
True
..
Internal Medicine
INTERNAL MEDICINE
True
..
Otolaryngology / Head and Neck
FACIAL PLASTIC SURGERY
15
False
UNSPECIFIED
None
..
Urology, Female Pelvic and Reconstructive Surgery
UROLOGY
True
..
Dermatology
DERMATOLOGY
True
..
Medicine, Infectious Disease
INFECTIOUS DISEASE
True
..
Neurology
NEUROLOGY
True
..
Family Medicine
FAMILY MEDICINE
True
..
Internal Medicine
NEPHROLOGY
15
False
UNSPECIFIED
None
..
Medicine, Gastroenterology
GASTROENTEROLOGY
True
..
Medicine, Gastroenterology
VASCULAR AND INTERVENTIONAL RADIOLOGY
38
False

..
Family Medicine
NEONATAL-PERINATAL MEDICINE
57
True
..
Family Medicine
FAMILY MEDICINE
True
..
Family Medicine
INFECTIOUS DISEASE
36
False
UNSPECIFIED
Family Medicine, Sports Medicine
..
Anesthesiology
ANATOMIC/CLINICAL PATHOLOGY
44
False
UNSPECIFIED
None
..
Anesthesiology
ANESTHESIOLOGY
True
..
Ophthalmology
OPHTHALMOLOGY
True
..
Ophthalmology
ANESTHESIOLOGY
52
True
..
Radiation Oncology
OTOLARYNGOLOGY-HEAD AND NECK SURGERY
26
False
UNSPECIFIED
None
..
Radiation Oncology
OPHTHALMOLOGY
39
False
UNSPECIFIED
None
..
Plastic Surgery
PLASTIC SURGERY
True
..
Urology
UROLOGY
True
..
Adult Psychiatry
PSYCHIATRY
True
..
Pediatrics
PEDIATRICS
True
..
Medicine, Hematology and Oncology
MEDICAL ONCOLOGY
65
True
..
Pediatrics, Endocrinology
PEDIATRIC ENDOCRINOLOGY
96
True
..
Internal Medicine
GERIATRIC MEDICINE (INTERNAL MEDICINE)
True
..
Anesthesiology
PAIN MEDICINE
22
False
UNSPECIFIED
None
..
Cytopathology
CYTOPATHOLOGY
True
..
Medicine, Gastroenterology
GASTROENTEROLOGY
True
..
Medicine, Rhe

..
Urology
UROLOGY
True
..
Anesthesiology
ANESTHESIOLOGY
True
..
Radiation Oncology
RADIATION ONCOLOGY
True
..
Family Medicine
FAMILY MEDICINE
True
..
Diagnostic and Interventional Radiology
DIAGNOSTIC RADIOLOGY
68
True
..
Otolaryngology / Head and Neck
OTOLARYNGOLOGY-HEAD AND NECK SURGERY
82
True
..
Medicine, Hospitalist
INTERNAL MEDICINE
42
False
UNSPECIFIED
Internal Medicine
..
Psychiatry
PSYCHIATRY
True
..
Medicine, Pulmonary Disease
PULMONARY & CRITICAL CARE MEDICINE
43
False
UNSPECIFIED
None
..
Obstetrics and Gynecology
OBSTETRICS & GYNECOLOGY
92
True
..
Medicine, Rheumatology
RHEUMATOLOGY
True
..
Ophthalmology
OPHTHALMOLOGY
True
..
Pathology
FAMILY MEDICINE
25
False
UNSPECIFIED
Pathology, Hematology
..
Pathology
ANATOMIC/CLINICAL PATHOLOGY
True
..
Pediatric Neurology
CHILD NEUROLOGY
65
True
..
Neurology
VASCULAR NEUROLOGY
True
..
Obstetrics and Gynecology, Obstetrics
OBSTETRICS & GYNECOLOGY
73
True
..
Anesthesiology
CRITICAL CARE MEDICINE (ANESTHESIOLOGY)
True
..
Obstetrics and 

True
..
Pediatrics, Hospitalist
PEDIATRICS
True
..
Orthopaedic Surgery, Adult Reconstructive Surgery - Joints
ADULT RECONSTRUCTIVE ORTHOPEDICS
49
True
..
Psychiatry
PSYCHIATRY
True
..
Radiation Oncology
RADIATION ONCOLOGY
True
..
Internal Medicine
INTERNAL MEDICINE
True
..
Ophthalmology
OPHTHALMOLOGY
True
..
Neurology
NEUROLOGY
True
..
Anesthesiology
PEDIATRIC ANESTHESIOLOGY (ANESTHESIOLOGY)
True
..
Obstetrics and Gynecology
OBSTETRICS & GYNECOLOGY
92
True
..
Medicine, Gastroenterology
GASTROENTEROLOGY
True
..
Adult Psychiatry
PSYCHIATRY
True
..
Child and Adolescent Psychiatry
CHILD & ADOLESCENT PSYCHIATRY
93
True
..
Urology, Pediatric Urology
UROLOGY
True
..
Pediatrics, Endocrinology
PEDIATRIC ENDOCRINOLOGY
96
True
..
Neurology
NEUROLOGY
True
..
Neurology
UNSPECIFIED
20
False
UNSPECIFIED
None
..
Medicine, Hematology and Oncology
HEMATOLOGY/MEDICAL ONCOLOGY
67
True
..
Child and Adolescent Psychiatry
CHILD & ADOLESCENT PSYCHIATRY
93
True
..
Medicine, Endocrinology - Diabetes and Metabol

Pediatrics, Nephrology
PEDIATRIC NEPHROLOGY
95
True
..
Internal Medicine
INTERNAL MEDICINE
True
..
Surgery
OTOLARYNGOLOGY-HEAD AND NECK SURGERY
True
..
Pediatrics, Rheumatology
PEDIATRIC RHEUMATOLOGY
96
True
..
Internal Medicine
INTERNAL MEDICINE
True
..
Adult Psychiatry
PSYCHIATRY
True
..
Medicine, Cardiology / Cardiovascular Disease
CLINICAL CARDIAC ELECTROPHYSIOLOGY
33
False
CARDIOVASCULAR DISEASE
None
..
Adult Psychiatry
PSYCHIATRY
True
..
Medicine, Pulmonary Disease
PULMONARY & CRITICAL CARE MEDICINE
43
False
UNSPECIFIED
Medicine, Critical Care Medicine
..
Psychiatry
PSYCHIATRY
True
..
Internal Medicine
UNSPECIFIED
21
False
UNSPECIFIED
None
..
Obstetrics and Gynecology, Obstetrics
DERMATOPATHOLOGY (DERMATOLOGY)
27
False
UNSPECIFIED
None
..
Anesthesiology
ADULT CARDIOTHORACIC ANESTHESIOLOGY (ANESTHESIOLOGY)
True
..
Neurology
NEUROLOGY
True
..
Neurology
OPHTHALMOLOGY
45
False
UNSPECIFIED
None
..
Neurology
NEUROLOGY
True
..
Medicine, Nephrology
FAMILY MEDICINE
46
True
..
Medicine, Ne

Family Medicine
FAMILY MEDICINE
True
..
Medicine, Hospitalist
UNSPECIFIED
12
False
UNSPECIFIED
Internal Medicine
..
Radiation Oncology
RADIATION ONCOLOGY
True
..
Ophthalmology
OPHTHALMOLOGY
True
..
Medicine, Hospitalist
INTERNAL MEDICINE
42
False
UNSPECIFIED
Internal Medicine
..
Ophthalmology
OPHTHALMOLOGY
True
..
Pediatrics, Hematology and Oncology
PEDIATRICS
True
..
Ophthalmology
OPHTHALMOLOGY
True
..
Diagnostic Radiology, Abdominal
DIAGNOSTIC RADIOLOGY
True
..
Anatomic Pathology and Clinical Pathology
ANATOMIC/CLINICAL PATHOLOGY
76
True
..
Medicine, Pulmonary Disease
PULMONARY & CRITICAL CARE MEDICINE
43
False
UNSPECIFIED
None
..
Medicine, Hospitalist
INTERNAL MEDICINE
42
False
UNSPECIFIED
Internal Medicine
..
Medicine, Hospitalist
INTERNAL MEDICINE
42
False
UNSPECIFIED
Internal Medicine
..
Pediatric Surgery
PEDIATRIC SURGERY (SURGERY)
True
..
Pediatrics, Hospitalist
PEDIATRICS
True
..
Pediatrics, Neonatal-Perinatal Medicine
NEONATAL-PERINATAL MEDICINE
True
..
Pediatrics, Infectious

In [503]:
that_all['KEEP']=match_list

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [505]:
hope = that_all[that_all.KEEP==True]

In [518]:
GAH = pd.merge(emails, hope, on='ME', how = 'right')

In [552]:
email_match = GAH[GAH.DOMAIN=='mednet.ucla.edu'].sort_values('PREFERRED', ascending=False)[['USERNAME','DOMAIN','PURPOSE_USG_DESC', 'name','POLO_MAILING_LINE_1','POLO_MAILING_LINE_2', 'location_address', 'location_address_city','POLO_CITY']]

In [535]:
GAH.columns

Index(['ME', 'EMAIL_ADDR', 'PURPOSE_USG_DESC', 'USERNAME', 'DOMAIN', 'name',
       'MIDDLE_NAME_x', 'SUFFIX_x', 'NICKNAME', 'TITLE', 'link', 'degree',
       'specialty', 'secondary_specialty', 'department', 'parent_org_name',
       'med_school', 'med_school_grad_year', 'residency_program', 'program',
       'location_name', 'location_address', 'location_address_city',
       'location_address_state', 'location_address_zip', 'phone', 'fax',
       'state_license_number', 'MAILING_NAME', 'MIDDLE_NAME_y', 'SUFFIX_y',
       'MD_DO_CODE', 'TELEPHONE_NUMBER', 'MEDSCHOOL_STATE',
       'MEDSCHOOL_GRAD_YEAR', 'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2',
       'POLO_CITY', 'POLO_STATE', 'POLO_ZIP', 'DESC_PRIMARY', 'DESC_SECONDARY',
       'DEGREE', 'KEEP'],
      dtype='object')

In [519]:
GAH = GAH.fillna('None')

In [554]:
email_none = GAH[GAH.EMAIL_ADDR=='None']

In [528]:
emails.groupby('DOMAIN').count().sort_values('ME', ascending=False)[0:50]

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME
DOMAIN,,,,
gmail.com,1107580,1107580,1107580,1107580
yahoo.com,383001,383001,383001,383001
hotmail.com,292194,292194,292194,292194
aol.com,234253,234253,234253,234253
comcast.net,71407,71407,71407,71407
msn.com,51123,51123,51123,51123
sbcglobal.net,30111,30111,30111,30111
bellsouth.net,19525,19525,19525,19525
mac.com,17472,17472,17472,17472


In [532]:
emails[emails['PURPOSE_USG_DESC']=='Preferred E-mail'].groupby('DOMAIN').count().sort_values('ME', ascending=False)[0:50]

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME
DOMAIN,,,,
gmail.com,224914,224914,224914,224914
yahoo.com,84494,84494,84494,84494
hotmail.com,70591,70591,70591,70591
aol.com,53468,53468,53468,53468
comcast.net,15724,15724,15724,15724
msn.com,11942,11942,11942,11942
sbcglobal.net,6453,6453,6453,6453
bellsouth.net,4226,4226,4226,4226
mayo.edu,3531,3531,3531,3531


In [531]:
emails.drop_duplicates('EMAIL_ADDR').groupby('DOMAIN').count().sort_values('ME', ascending=False)[0:50]

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME
DOMAIN,,,,
gmail.com,360134,360134,360134,360134
yahoo.com,116099,116099,116099,116099
hotmail.com,104675,104675,104675,104675
aol.com,74881,74881,74881,74881
comcast.net,26327,26327,26327,26327
msn.com,17765,17765,17765,17765
sbcglobal.net,8976,8976,8976,8976
bellsouth.net,6028,6028,6028,6028
cox.net,5323,5323,5323,5323


In [538]:
GAH

,ME,EMAIL_ADDR,PURPOSE_USG_DESC,USERNAME,DOMAIN,name,MIDDLE_NAME_x,SUFFIX_x,NICKNAME,TITLE,...,MEDSCHOOL_GRAD_YEAR,POLO_MAILING_LINE_1,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,DESC_PRIMARY,DESC_SECONDARY,DEGREE,KEEP
0,00106150371,jmccormick22@bellsouth.net,Morning Rounds ...,jmccormick22,bellsouth.net,Justin McCormick,,,,,...,2015,None,None,None,None,None,OTOLARYNGOLOGY-HEAD AND NECK SURGERY,UNSPECIFIED,MD,True
1,00106150371,jmccormick22@bellsouth.net,Morning Rounds Daily Edition ...,jmccormick22,bellsouth.net,Justin McCormick,,,,,...,2015,None,None,None,None,None,OTOLARYNGOLOGY-HEAD AND NECK SURGERY,UNSPECIFIED,MD,True
2,00106150371,jmccormick22@bellsouth.net,No Usage,jmccormick22,bellsouth.net,Justin McCormick,,,,,...,2015,None,None,None,None,None,OTOLARYNGOLOGY-HEAD AND NECK SURGERY,UNSPECIFIED,MD,True
3,00106150371,jpm603@jagmail.southalabama.edu,No Usage,jpm603,jagmail.southalabama.edu,Justin McCormick,,,,,...,2015,None,None,None,None,None,OTOLARYNGOLOGY-HEAD AND NECK SURGERY,UNSPECIFIED,MD,True
4,00106150371,jpm603@jagmail.southalabama.edu,Preferred E-mail,jpm603,jagmail.southalabama.edu,Justin McCormick,,,,,...,2015,None,None,None,None,None,OTOLARYNGOLOGY-HEAD AND NECK SURGERY,UNSPECIFIED,MD,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6773,04115151440,None,None,None,None,Colleen Yen,,,,,...,2015,BOX 1010,1 GUSTAVE L LEVY PL FL 12,NEW YORK,NY,10029,OBSTETRIC ANESTHESIOLOGY (ANESTHESIOLOGY),UNSPECIFIED,MD,True
6774,46201660084,None,None,None,None,David T. Yu,T.,,,,...,1966,None,230 12TH ST,SANTA MONICA,CA,90402,RHEUMATOLOGY,ALLERGY AND IMMUNOLOGY,MD,True
6775,01601011308,None,None,None,None,Tim Yuen,,,,,...,2001,OLIVE VIEW MEDICAL CENTER,14445 OLIVE VIEW DR RM 3A115,SYLMAR,CA,91342,ANESTHESIOLOGY,UNSPECIFIED,MD,True
6776,00576070951,None,None,None,None,Tannaz Zahirpour,,,,,...,2007,None,2925 SYCAMORE DR STE 204,SIMI VALLEY,CA,93065,FAMILY MEDICINE,UNSPECIFIED,DO,True


In [553]:
email_mismatch = GAH[(GAH.DOMAIN!='mednet.ucla.edu')&(GAH.DOMAIN!='None')]

In [546]:
GAH['PREFERRED']=GAH.PURPOSE_USG_DESC=='Preferred E-mail'

In [558]:
with pd.ExcelWriter("../../Data/Email/UCLA_EMAILS.xlsx") as writer:  
        email_none.to_excel(writer, sheet_name='No Email', index=False)
        email_match.to_excel(writer, sheet_name='UCLA', index=False)
        email_mismatch.to_excel(writer, sheet_name='Email Mismatch', index=False)

In [ ]:
with pd.ExcelWriter("../../Data/Email/HOSPITAL_EMAILS.xlsx") as writer:  
        email_none.to_excel(writer, sheet_name='No Email', index=False)
        email_match.to_excel(writer, sheet_name='MAYO', index=False)
        email_mismatch.to_excel(writer, sheet_name='Email Mismatch', index=False)